# 第七章 型態、方法與介面 - Part 3: 介面系統

本部分將深入探討 Go 語言的介面系統，這是 Go 最強大和優雅的特性之一。Go 的介面採用隱式實現，提供了型態安全的多型和抽象機制。

## 介面快速入門

Go 的介面是其型態系統的核心特性，提供了強大而優雅的抽象機制。Go 的介面是隱式的，不需要明確宣告實現關係。

### 介面定義語法
**介面定義語法**:
```go
type InterfaceName interface {
    Method1(param Type) ReturnType
    Method2() (Type1, Type2)
}
```

### 基本特性
- **隱式實現**: 型態不需要明確宣告實現介面
- **方法集合**: 介面定義一組方法的集合
- **Duck Typing**: \"如果它走起來像鴨子，叫起來像鴨子，那就是鴨子\"
- **零值**: 介面的零值是 nil

### 介面的優勢
- **解耦**: 降低程式碼間的耦合度
- **測試**: 易於 mock 和測試
- **多型**: 同一介面可以有多種實現
- **抽象**: 隱藏實現細節，專注於行為

In [ ]:
package main

import "fmt"

// ======================================================================
// 基本介面定義和實現
// ======================================================================

// 定義 Writer 介面
type Writer interface {
    Write(data []byte) (int, error)
}

// 定義 Reader 介面
type Reader interface {
    Read(data []byte) (int, error)
}

// 定義 Closer 介面
type Closer interface {
    Close() error
}

// 實現 Writer 介面的結構體
type FileWriter struct {
    filename string
    content  []byte
}

// FileWriter 實現 Write 方法，因此自動實現了 Writer 介面
func (fw *FileWriter) Write(data []byte) (int, error) {
    fw.content = append(fw.content, data...)
    return len(data), nil
}

func (fw *FileWriter) GetContent() string {
    return string(fw.content)
}

// 另一個實現 Writer 介面的結構體
type ConsoleWriter struct {
    prefix string
}

// ConsoleWriter 也實現 Write 方法
func (cw ConsoleWriter) Write(data []byte) (int, error) {
    fmt.Printf("%s: %s", cw.prefix, string(data))
    return len(data), nil
}

// 實現多個介面的結構體
type BufferedFile struct {
    name    string
    buffer  []byte
    isOpen  bool
}

func (bf *BufferedFile) Write(data []byte) (int, error) {
    if !bf.isOpen {
        return 0, fmt.Errorf("file is closed")
    }
    bf.buffer = append(bf.buffer, data...)
    return len(data), nil
}

func (bf *BufferedFile) Read(data []byte) (int, error) {
    if !bf.isOpen {
        return 0, fmt.Errorf("file is closed")
    }
    n := copy(data, bf.buffer)
    return n, nil
}

func (bf *BufferedFile) Close() error {
    bf.isOpen = false
    return nil
}

func (bf *BufferedFile) Open() error {
    bf.isOpen = true
    return nil
}

// ======================================================================
// 介面作為參數的函式
// ======================================================================

// 接受 Writer 介面作為參數
func WriteMessage(w Writer, message string) {
    data := []byte(message)
    n, err := w.Write(data)
    if err != nil {
        fmt.Printf("Error writing: %v\n", err)
    } else {
        fmt.Printf("Successfully wrote %d bytes\n", n)
    }
}

// 接受多個介面作為參數
func ProcessFile(w Writer, r Reader, closer Closer) {
    // 讀取資料
    buffer := make([]byte, 1024)
    n, err := r.Read(buffer)
    if err != nil {
        fmt.Printf("Error reading: %v\n", err)
        return
    }
    
    // 寫入資料
    _, err = w.Write(buffer[:n])
    if err != nil {
        fmt.Printf("Error writing: %v\n", err)
        return
    }
    
    // 關閉資源
    err = closer.Close()
    if err != nil {
        fmt.Printf("Error closing: %v\n", err)
    } else {
        fmt.Println("Resource closed successfully")
    }
}

func main() {
    fmt.Println("=== 基本介面使用示例 ===")
    
    // 創建不同的 Writer 實現
    fileWriter := &FileWriter{filename: "test.txt"}
    consoleWriter := ConsoleWriter{prefix: "LOG"}
    
    // 使用介面：同一個函式可以接受不同的實現
    WriteMessage(fileWriter, "Hello from FileWriter\n")    // 寫入到檔案
    WriteMessage(consoleWriter, "Hello from Console\n")    // 輸出到控制台
    
    // 檢查檔案內容
    fmt.Printf("FileWriter content: %s", fileWriter.GetContent())  // 輸出: FileWriter content: Hello from FileWriter
    
    fmt.Println("\n=== 多介面實現示例 ===")
    
    // BufferedFile 實現了三個介面
    bufferedFile := &BufferedFile{name: "buffer.txt"}
    bufferedFile.Open()
    
    // 先寫入一些資料
    WriteMessage(bufferedFile, "Hello World!\n")
    
    // 作為多個介面使用
    ProcessFile(bufferedFile, bufferedFile, bufferedFile)
    
    fmt.Println("\n=== 介面變數示例 ===")
    
    // 介面變數可以持有任何實現該介面的值
    var writer Writer
    
    writer = fileWriter           // 指派 FileWriter
    fmt.Printf("writer type: %T\n", writer)  // 輸出: writer type: *main.FileWriter
    
    writer = consoleWriter        // 指派 ConsoleWriter
    fmt.Printf("writer type: %T\n", writer)  // 輸出: writer type: main.ConsoleWriter
    
    writer = bufferedFile         // 指派 BufferedFile
    fmt.Printf("writer type: %T\n", writer)  // 輸出: writer type: *main.BufferedFile
    
    // 使用目前的 writer
    WriteMessage(writer, "Using interface variable\n")
    
    fmt.Println("\n=== 空介面檢查 ===")
    
    var emptyWriter Writer
    if emptyWriter == nil {
        fmt.Println("emptyWriter is nil")  // 會執行這行
    }
    
    fmt.Println("\n=== 介面的優勢展示 ===")
    fmt.Println("1. 一個函式可以接受多種實現")
    fmt.Println("2. 易於測試：可以建立 mock 實現")
    fmt.Println("3. 鬆耦合：不依賴具體實現")
    fmt.Println("4. 擴展性：新增實現不需修改現有程式碼")
}

## 介面是型態安全的鴨子定型

Go 實現了型態安全的鴨子定型系統。與動態語言不同，Go 在編譯時就能確定型態是否實現了介面。

### 鴨子定型的原則
- 如果一個型態實現了介面的所有方法，它就實現了該介面
- 不需要明確的 `implements` 關鍵字
- 編譯時檢查，執行時安全

### 與動態鴨子定型的區別
```go
// Go: 編譯時檢查
type Writer interface {
    Write([]byte) (int, error)
}

// 任何有 Write 方法的型態都自動實現 Writer
```

### 型態安全的保證
- **編譯時驗證**: 確保型態確實實現了所需的方法
- **方法簽章檢查**: 確保方法簽章完全匹配
- **執行時效率**: 沒有動態方法查找的開銷

In [ ]:
package main

import "fmt"

// ======================================================================
// 鴨子定型：多種不同的型態實現相同的介面
// ======================================================================

// 定義 Speaker 介面
type Speaker interface {
    Speak() string
}

// 動物型態
type Dog struct {
    Name string
}

func (d Dog) Speak() string {
    return d.Name + " says: Woof!"
}

type Cat struct {
    Name string
}

func (c Cat) Speak() string {
    return c.Name + " says: Meow!"
}

type Duck struct {
    Name string
}

func (duck Duck) Speak() string {
    return duck.Name + " says: Quack!"
}

// 非動物型態也可以實現相同介面
type Robot struct {
    Model string
}

func (r Robot) Speak() string {
    return "Robot " + r.Model + " says: Beep boop!"
}

type Human struct {
    Name string
}

func (h Human) Speak() string {
    return "Human " + h.Name + " says: Hello!"
}

// ======================================================================
// 編譯時型態檢查示例
// ======================================================================

// 定義 Formatter 介面
type Formatter interface {
    Format() string
}

// 正確實現 Formatter 介面
type TextFormatter struct {
    content string
}

func (tf TextFormatter) Format() string {  // 方法簽章正確
    return "Text: " + tf.content
}

// 錯誤的實現（方法簽章不匹配）
type BadFormatter struct {
    content string
}

// 這個方法簽章不匹配 Formatter 介面
// func (bf BadFormatter) Format(prefix string) string {  // 錯誤：參數不匹配
//     return prefix + bf.content
// }

// 另一個錯誤的實現（回傳型態不匹配）
// func (bf BadFormatter) Format() int {  // 錯誤：回傳型態不匹配
//     return len(bf.content)  
// }

// ======================================================================
// 介面相容性檢查
// ======================================================================

// 定義多個相關介面
type Reader interface {
    Read() string
}

type Writer interface {
    Write(data string)
}

type ReadWriter interface {
    Read() string
    Write(data string)
}

// 實現多個介面的型態
type File struct {
    content string
}

func (f *File) Read() string {
    return f.content
}

func (f *File) Write(data string) {
    f.content = data
}

// File 自動實現了 Reader, Writer, 和 ReadWriter 介面

// ======================================================================
// 使用鴨子定型的函式
// ======================================================================

// 接受 Speaker 介面的函式
func MakeSpeak(speaker Speaker) {
    fmt.Println(speaker.Speak())
}

// 接受多個 Speaker 的函式
func AnimalChorus(speakers []Speaker) {
    fmt.Println("=== Animal Chorus ===")
    for i, speaker := range speakers {
        fmt.Printf("%d: %s\n", i+1, speaker.Speak())
    }
}

// 檢查型態是否實現介面的編譯時驗證
func VerifyInterface() {
    // 編譯時檢查：這些賦值如果型態不匹配會導致編譯錯誤
    var _ Speaker = Dog{}      // Dog 實現了 Speaker
    var _ Speaker = Cat{}      // Cat 實現了 Speaker  
    var _ Speaker = Duck{}     // Duck 實現了 Speaker
    var _ Speaker = Robot{}    // Robot 實現了 Speaker
    var _ Speaker = Human{}    // Human 實現了 Speaker
    
    var _ Formatter = TextFormatter{}  // TextFormatter 實現了 Formatter
    // var _ Formatter = BadFormatter{}   // 這行會編譯錯誤
    
    var _ Reader = (*File)(nil)     // File 實現了 Reader
    var _ Writer = (*File)(nil)     // File 實現了 Writer
    var _ ReadWriter = (*File)(nil) // File 實現了 ReadWriter
    
    fmt.Println("所有介面實現都通過了編譯時檢查")
}

func main() {
    fmt.Println("=== 鴨子定型示例 ===")
    
    // 創建不同型態的實例
    dog := Dog{Name: "Buddy"}
    cat := Cat{Name: "Whiskers"}
    duck := Duck{Name: "Donald"}
    robot := Robot{Model: "R2D2"}
    human := Human{Name: "Alice"}
    
    // 同一個函式可以處理所有實現 Speaker 介面的型態
    MakeSpeak(dog)     // 輸出: Buddy says: Woof!
    MakeSpeak(cat)     // 輸出: Whiskers says: Meow!
    MakeSpeak(duck)    // 輸出: Donald says: Quack!
    MakeSpeak(robot)   // 輸出: Robot R2D2 says: Beep boop!
    MakeSpeak(human)   // 輸出: Human Alice says: Hello!
    
    // 將不同型態放在同一個切片中
    speakers := []Speaker{dog, cat, duck, robot, human}
    AnimalChorus(speakers)
    
    fmt.Println("\n=== 編譯時型態安全 ===")
    
    // 編譯時驗證介面實現
    VerifyInterface()
    
    // 使用正確的實現
    formatter := TextFormatter{content: "Hello World"}
    fmt.Printf("Formatted text: %s\n", formatter.Format())  // 輸出: Formatted text: Text: Hello World
    
    fmt.Println("\n=== 多介面實現示例 ===")
    
    file := &File{}
    
    // File 可以作為不同介面使用
    var reader Reader = file
    var writer Writer = file  
    var readWriter ReadWriter = file
    
    // 寫入資料
    writer.Write("Hello, Go interfaces!")
    
    // 讀取資料
    content := reader.Read()
    fmt.Printf("File content: %s\n", content)  // 輸出: File content: Hello, Go interfaces!
    
    // 作為 ReadWriter 使用
    readWriter.Write("Updated content")
    updatedContent := readWriter.Read()
    fmt.Printf("Updated content: %s\n", updatedContent)  // 輸出: Updated content: Updated content
    
    fmt.Println("\n=== 鴨子定型的優勢 ===")
    fmt.Println("1. 編譯時型態安全：確保方法存在且簽章正確")
    fmt.Println("2. 無需明確宣告：自動識別介面實現")
    fmt.Println("3. 靈活性：新型態可以自然地實現現有介面")
    fmt.Println("4. 效能：沒有動態方法查找的開銷")
    fmt.Println("5. 可測試性：容易建立 mock 實現")
}

## 內嵌介面

介面可以嵌入其他介面，形成更複雜的介面組合：

```go
type Reader interface {
    Read([]byte) (int, error)
}

type Writer interface {
    Write([]byte) (int, error)
}

type ReadWriter interface {
    Reader  // 嵌入 Reader 介面
    Writer  // 嵌入 Writer 介面
}
```

### 介面嵌入的特性
- **方法集合合併**: 嵌入介面的所有方法都成為新介面的方法
- **無名衝突**: 如果有同名方法，簽章必須完全相同
- **層次化設計**: 可以建立分層的介面架構
- **向後相容**: 小介面組合成大介面

### 使用場景
- **標準庫模式**: io.ReadWriter, io.ReadWriteCloser
- **能力組合**: 將不同的能力組合成複合介面
- **階段式設計**: 先定義小介面，再組合成大介面

In [ ]:
package main

import "fmt"

// ======================================================================
// 基礎介面定義
// ======================================================================

// 基礎 I/O 介面
type Reader interface {
    Read(data []byte) (int, error)
}

type Writer interface {
    Write(data []byte) (int, error)
}

type Closer interface {
    Close() error
}

// ======================================================================
// 內嵌介面：組合基礎介面
// ======================================================================

// ReadWriter 嵌入了 Reader 和 Writer 介面
type ReadWriter interface {
    Reader  // 嵌入 Reader，包含 Read 方法
    Writer  // 嵌入 Writer，包含 Write 方法
}

// ReadWriteCloser 嵌入了三個介面
type ReadWriteCloser interface {
    Reader  // Read 方法
    Writer  // Write 方法  
    Closer  // Close 方法
}

// ======================================================================
// 更複雜的介面嵌入
// ======================================================================

// 定義各種能力介面
type Validator interface {
    Validate() error
}

type Serializer interface {
    Serialize() ([]byte, error)
}

type Deserializer interface {
    Deserialize([]byte) error
}

// 組合序列化能力
type SerializationHandler interface {
    Serializer    // 序列化能力
    Deserializer  // 反序列化能力
}

// 完整的資料處理器介面
type DataProcessor interface {
    ReadWriter           // I/O 能力
    Validator           // 驗證能力
    SerializationHandler // 序列化能力
}

// ======================================================================
// 層次化介面設計
// ======================================================================

// 基礎連線介面
type Connection interface {
    Connect() error
    Disconnect() error
    IsConnected() bool
}

// 可配置的連線
type ConfigurableConnection interface {
    Connection  // 基礎連線能力
    SetTimeout(seconds int)
    GetTimeout() int
}

// 安全連線
type SecureConnection interface {
    ConfigurableConnection  // 包含配置能力
    EnableSSL() error
    DisableSSL() error
}

// 高級連線（包含所有能力）
type AdvancedConnection interface {
    SecureConnection  // 包含安全連線的所有能力
    ReadWriteCloser   // 包含 I/O 能力
    GetStats() ConnectionStats
}

type ConnectionStats struct {
    BytesRead    int64
    BytesWritten int64
    Uptime       int64
}

// ======================================================================
// 實現複合介面
// ======================================================================

// Buffer 實現多個基礎介面
type Buffer struct {
    data      []byte
    readPos   int
    writePos  int
    closed    bool
}

func (b *Buffer) Read(data []byte) (int, error) {
    if b.closed {
        return 0, fmt.Errorf("buffer is closed")
    }
    n := copy(data, b.data[b.readPos:b.writePos])
    b.readPos += n
    return n, nil
}

func (b *Buffer) Write(data []byte) (int, error) {
    if b.closed {
        return 0, fmt.Errorf("buffer is closed")
    }
    b.data = append(b.data, data...)
    n := len(data)
    b.writePos += n
    return n, nil
}

func (b *Buffer) Close() error {
    b.closed = true
    return nil
}

// Buffer 自動實現了 Reader, Writer, Closer, ReadWriter, ReadWriteCloser

// ======================================================================
// 實現複雜的組合介面
// ======================================================================

type Document struct {
    content string
    valid   bool
}

func (d *Document) Read(data []byte) (int, error) {
    contentBytes := []byte(d.content)
    n := copy(data, contentBytes)
    return n, nil
}

func (d *Document) Write(data []byte) (int, error) {
    d.content = string(data)
    d.valid = false  // 寫入後需要重新驗證
    return len(data), nil
}

func (d *Document) Validate() error {
    if len(d.content) == 0 {
        return fmt.Errorf("document content is empty")
    }
    d.valid = true
    return nil
}

func (d *Document) Serialize() ([]byte, error) {
    if !d.valid {
        return nil, fmt.Errorf("document is not valid")
    }
    return []byte(d.content), nil
}

func (d *Document) Deserialize(data []byte) error {
    d.content = string(data)
    d.valid = false
    return d.Validate()
}

// Document 實現了多個組合介面

// ======================================================================
// 使用內嵌介面的函式
// ======================================================================

// 接受 ReadWriter 介面
func CopyData(rw ReadWriter, source string) error {
    // 寫入資料
    _, err := rw.Write([]byte(source))
    if err != nil {
        return err
    }
    
    // 讀取資料進行驗證
    buffer := make([]byte, 1024)
    n, err := rw.Read(buffer)
    if err != nil {
        return err
    }
    
    fmt.Printf("Copied and verified %d bytes: %s\n", n, string(buffer[:n]))
    return nil
}

// 接受 DataProcessor 介面（最複雜的組合介面）
func ProcessDocument(processor DataProcessor, content string) error {
    // 寫入內容
    _, err := processor.Write([]byte(content))
    if err != nil {
        return err
    }
    
    // 驗證
    err = processor.Validate()
    if err != nil {
        return fmt.Errorf("validation failed: %v", err)
    }
    
    // 序列化
    data, err := processor.Serialize()
    if err != nil {
        return fmt.Errorf("serialization failed: %v", err)
    }
    
    fmt.Printf("Processed document: %d bytes serialized\n", len(data))
    return nil
}

func main() {
    fmt.Println("=== 內嵌介面示例 ===")
    
    // Buffer 實現了多個介面
    buffer := &Buffer{
        data: make([]byte, 0),
    }
    
    // 作為 ReadWriter 使用
    var rw ReadWriter = buffer
    err := CopyData(rw, "Hello, embedded interfaces!")
    if err != nil {
        fmt.Printf("Error: %v\n", err)
    }
    
    // 作為 ReadWriteCloser 使用
    var rwc ReadWriteCloser = buffer
    rwc.Close()  // 關閉 buffer
    
    fmt.Println("\n=== 複雜介面組合示例 ===")
    
    // Document 實現了 DataProcessor 介面
    doc := &Document{}
    
    // 作為 DataProcessor 使用（包含所有嵌入介面的能力）
    var processor DataProcessor = doc
    
    err = ProcessDocument(processor, "This is a sample document content.")
    if err != nil {
        fmt.Printf("Error processing document: %v\n", err)
    }
    
    fmt.Println("\n=== 介面層次展示 ===")
    
    // 展示 Document 實現了哪些介面
    var _ Reader = doc              // 基礎介面
    var _ Writer = doc              // 基礎介面
    var _ Validator = doc           // 基礎介面
    var _ Serializer = doc          // 基礎介面
    var _ Deserializer = doc        // 基礎介面
    var _ ReadWriter = doc          // 組合介面
    var _ SerializationHandler = doc // 組合介面
    var _ DataProcessor = doc       // 最高層組合介面
    
    fmt.Println("Document 成功實現了所有相關介面")
    
    fmt.Println("\n=== 介面向下相容示例 ===")
    
    // 高層介面可以向下相容使用
    var reader Reader = doc         // DataProcessor 可以當作 Reader
    var writer Writer = doc         // DataProcessor 可以當作 Writer
    var validator Validator = doc   // DataProcessor 可以當作 Validator
    
    // 分別使用不同能力
    content := []byte("Updated content")
    writer.Write(content)  // 寫入
    
    err = validator.Validate()  // 驗證
    if err != nil {
        fmt.Printf("Validation error: %v\n", err)
    } else {
        fmt.Println("Document validation passed")
    }
    
    // 讀取
    buffer2 := make([]byte, 100)
    n, _ := reader.Read(buffer2)
    fmt.Printf("Read %d bytes: %s\n", n, string(buffer2[:n]))  // 輸出: Read 15 bytes: Updated content
    
    fmt.Println("\n=== 內嵌介面的優勢 ===")
    fmt.Println("1. 組合優於繼承：通過組合建立複雜介面")
    fmt.Println("2. 分層設計：從簡單介面組合出複雜介面")
    fmt.Println("3. 向下相容：複雜介面可以當作簡單介面使用")
    fmt.Println("4. 清晰的職責劃分：每個基礎介面負責特定功能")
    fmt.Println("5. 易於測試：可以分別 mock 不同層次的介面")
}

## 接收介面，回傳 struct

這是 Go 的一個重要設計原則：函式應該接受介面作為參數，但回傳具體的型態。

### 設計原則
- **接受介面**: 讓函式更靈活，可以接受任何實現該介面的型態
- **回傳具體型態**: 讓呼叫者可以存取完整的 API

```go
// 好的設計
func NewProcessor(w Writer) *FileProcessor {
    // 接受介面，回傳具體型態
    return &FileProcessor{writer: w}
}
```

### 為什麼這樣設計
- **靈活性**: 接受介面讓函式可以處理多種實現
- **完整性**: 回傳具體型態讓呼叫者獲得完整功能
- **向前相容**: 具體型態可以增加新方法而不破壞介面
- **測試友好**: 容易建立 mock 實現

In [ ]:
package main

import "fmt"

// ======================================================================
// 介面定義
// ======================================================================

type Logger interface {
    Log(message string)
}

type Storage interface {
    Store(key string, value interface{}) error
    Retrieve(key string) (interface{}, error)
}

type Notifier interface {
    Notify(message string) error
}

// ======================================================================
// 具體實現
// ======================================================================

type ConsoleLogger struct {
    prefix string
}

func (c *ConsoleLogger) Log(message string) {
    fmt.Printf("[%s] %s\n", c.prefix, message)
}

type MemoryStorage struct {
    data map[string]interface{}
}

func (m *MemoryStorage) Store(key string, value interface{}) error {
    if m.data == nil {
        m.data = make(map[string]interface{})
    }
    m.data[key] = value
    return nil
}

func (m *MemoryStorage) Retrieve(key string) (interface{}, error) {
    if m.data == nil {
        return nil, fmt.Errorf("storage not initialized")
    }
    value, exists := m.data[key]
    if !exists {
        return nil, fmt.Errorf("key not found: %s", key)
    }
    return value, nil
}

type EmailNotifier struct {
    smtpServer string
}

func (e *EmailNotifier) Notify(message string) error {
    fmt.Printf("Sending email via %s: %s\n", e.smtpServer, message)
    return nil
}

// ======================================================================
// 好的設計：接受介面，回傳具體型態
// ======================================================================

// UserService 是具體的型態
type UserService struct {
    logger   Logger    // 接受介面
    storage  Storage   // 接受介面
    notifier Notifier  // 接受介面
    userCount int      // 具體型態獨有的欄位
}

// 工廠函式：接受介面，回傳具體型態
func NewUserService(logger Logger, storage Storage, notifier Notifier) *UserService {
    return &UserService{
        logger:    logger,
        storage:   storage,
        notifier:  notifier,
        userCount: 0,
    }
}

// UserService 的方法（具體型態提供完整 API）
func (us *UserService) CreateUser(name, email string) error {
    us.logger.Log(fmt.Sprintf("Creating user: %s", name))
    
    user := map[string]string{
        "name":  name,
        "email": email,
    }
    
    err := us.storage.Store(name, user)
    if err != nil {
        us.logger.Log(fmt.Sprintf("Failed to store user: %v", err))
        return err
    }
    
    us.userCount++  // 具體型態獨有的功能
    
    err = us.notifier.Notify(fmt.Sprintf("New user created: %s", name))
    if err != nil {
        us.logger.Log(fmt.Sprintf("Failed to send notification: %v", err))
    }
    
    return nil
}

func (us *UserService) GetUser(name string) (map[string]string, error) {
    us.logger.Log(fmt.Sprintf("Retrieving user: %s", name))
    
    data, err := us.storage.Retrieve(name)
    if err != nil {
        return nil, err
    }
    
    user, ok := data.(map[string]string)
    if !ok {
        return nil, fmt.Errorf("invalid user data format")
    }
    
    return user, nil
}

// 具體型態獨有的方法（介面無法提供的功能）
func (us *UserService) GetUserCount() int {
    return us.userCount
}

func (us *UserService) GetStatistics() ServiceStats {
    return ServiceStats{
        UserCount:    us.userCount,
        ServiceName:  "UserService",
        Dependencies: []string{"Logger", "Storage", "Notifier"},
    }
}

type ServiceStats struct {
    UserCount    int
    ServiceName  string
    Dependencies []string
}

// ======================================================================
// 壞的設計對比（僅作說明，不要這樣做）
// ======================================================================

// 壞的設計：回傳介面而不是具體型態
/*
type UserManager interface {
    CreateUser(name, email string) error
    GetUser(name string) (map[string]string, error)
}

// 這樣設計的問題：
// 1. 呼叫者無法存取 GetUserCount() 等擴展方法
// 2. 後續添加新方法會破壞介面相容性
// 3. 失去了具體型態的靈活性
func NewUserManager(logger Logger, storage Storage) UserManager {
    return &UserService{...}
}
*/

// ======================================================================
// 測試友好的設計
// ======================================================================

// Mock 實現用於測試
type MockLogger struct {
    messages []string
}

func (m *MockLogger) Log(message string) {
    m.messages = append(m.messages, message)
}

func (m *MockLogger) GetMessages() []string {
    return m.messages
}

type MockStorage struct {
    data map[string]interface{}
    fail bool
}

func (m *MockStorage) Store(key string, value interface{}) error {
    if m.fail {
        return fmt.Errorf("mock storage error")
    }
    if m.data == nil {
        m.data = make(map[string]interface{})
    }
    m.data[key] = value
    return nil
}

func (m *MockStorage) Retrieve(key string) (interface{}, error) {
    if m.fail {
        return nil, fmt.Errorf("mock storage error")
    }
    return m.data[key], nil
}

type MockNotifier struct {
    notifications []string
}

func (m *MockNotifier) Notify(message string) error {
    m.notifications = append(m.notifications, message)
    return nil
}

func main() {
    fmt.Println("=== 接受介面，回傳具體型態示例 ===")
    
    // 創建具體實現
    logger := &ConsoleLogger{prefix: "USER_SVC"}
    storage := &MemoryStorage{}
    notifier := &EmailNotifier{smtpServer: "smtp.example.com"}
    
    // 好的設計：接受介面，回傳具體型態
    userService := NewUserService(logger, storage, notifier)
    
    // 使用服務
    err := userService.CreateUser("Alice", "alice@example.com")
    if err != nil {
        fmt.Printf("Error creating user: %v\n", err)
        return
    }
    
    err = userService.CreateUser("Bob", "bob@example.com")
    if err != nil {
        fmt.Printf("Error creating user: %v\n", err)
        return
    }
    
    // 獲取用戶
    user, err := userService.GetUser("Alice")
    if err != nil {
        fmt.Printf("Error getting user: %v\n", err)
    } else {
        fmt.Printf("Retrieved user: %+v\n", user)  // 輸出: Retrieved user: map[email:alice@example.com name:Alice]
    }
    
    // 使用具體型態的額外功能
    fmt.Printf("Total users: %d\n", userService.GetUserCount())  // 輸出: Total users: 2
    
    stats := userService.GetStatistics()
    fmt.Printf("Service statistics: %+v\n", stats)  // 輸出服務統計資訊
    
    fmt.Println("\n=== 測試友好設計示例 ===")
    
    // 使用 mock 實現進行測試
    mockLogger := &MockLogger{}
    mockStorage := &MockStorage{}
    mockNotifier := &MockNotifier{}
    
    // 同樣的工廠函式，不同的實現
    testService := NewUserService(mockLogger, mockStorage, mockNotifier)
    
    // 執行測試操作
    err = testService.CreateUser("TestUser", "test@example.com")
    if err != nil {
        fmt.Printf("Test failed: %v\n", err)
    }
    
    // 驗證 mock 的行為
    fmt.Printf("Mock logger captured %d messages\n", len(mockLogger.GetMessages()))  // 輸出: Mock logger captured 1 messages
    fmt.Printf("First log message: %s\n", mockLogger.GetMessages()[0])  // 輸出: First log message: Creating user: TestUser
    
    fmt.Println("\n=== 設計原則總結 ===")
    fmt.Println("好的設計模式：")
    fmt.Println("✓ 函式參數接受介面 - 提供靈活性")
    fmt.Println("✓ 函式回傳具體型態 - 提供完整功能")
    fmt.Println("✓ 呼叫者可以存取所有方法")
    fmt.Println("✓ 容易進行單元測試")
    fmt.Println("✓ 向前相容性良好")
    
    fmt.Println("\n壞的設計模式：")
    fmt.Println("✗ 回傳介面 - 限制功能存取")
    fmt.Println("✗ 難以擴展新功能")
    fmt.Println("✗ 破壞向前相容性")
    
    fmt.Println("\n=== 實際應用場景 ===")
    fmt.Println("1. 工廠函式：接受依賴介面，回傳服務實例")
    fmt.Println("2. 建構函式：接受配置介面，回傳具體物件")
    fmt.Println("3. 服務層：接受 repository 介面，回傳 service 實例")
    fmt.Println("4. 中間件：接受處理器介面，回傳包裝後的處理器")
}

## 介面與 nil

Go 中的介面值包含兩個部分：型態和值。理解這個結構對於正確處理 nil 很重要：

### 介面的內部結構
- **型態資訊**: 具體型態的資訊
- **資料指標**: 指向具體值的指標

### nil 介面的陷阱
```go
var w Writer        // nil 介面
var fw *FileWriter  // nil 指標
w = fw              // w 不是 nil！它有型態資訊
```

### 正確檢查介面是否為 nil
- 檢查介面本身：`if w == nil`
- 檢查介面值：使用反射或型態斷言
- 避免將 nil 指標賦值給介面

In [ ]:
package main

import (
    "fmt"
    "reflect"
)

type Writer interface {
    Write(data []byte) (int, error)
}

type FileWriter struct {
    filename string
}

func (fw *FileWriter) Write(data []byte) (int, error) {
    if fw == nil {
        return 0, fmt.Errorf("FileWriter is nil")
    }
    fmt.Printf("Writing to %s: %s", fw.filename, string(data))
    return len(data), nil
}

func main() {
    fmt.Println("=== 介面 nil 陷阱示例 ===")
    
    var w Writer           // nil 介面
    var fw *FileWriter     // nil 指標
    
    fmt.Printf("w == nil: %t\n", w == nil)     // 輸出: true
    fmt.Printf("fw == nil: %t\n", fw == nil)   // 輸出: true
    
    // 陷阱：將 nil 指標賦值給介面
    w = fw
    fmt.Printf("After w = fw, w == nil: %t\n", w == nil)  // 輸出: false！
    
    // 檢查介面的具體值
    if w != nil {
        val := reflect.ValueOf(w)
        fmt.Printf("Interface is not nil, but value is nil: %t\n", val.IsNil())  // 輸出: true
    }
    
    // 嘗試使用會 panic
    _, err := w.Write([]byte("test"))
    fmt.Printf("Write error: %v\n", err)  // 輸出: Write error: FileWriter is nil
    
    fmt.Println("\n=== 正確的 nil 檢查 ===")
    
    // 正確的檢查方式
    if w == nil {
        fmt.Println("Interface is nil")
    } else if reflect.ValueOf(w).IsNil() {
        fmt.Println("Interface is not nil, but holds a nil pointer")  // 會執行這行
    } else {
        fmt.Println("Interface holds a valid value")
    }
}

## 空介面沒有任何意思

空介面 `interface{}` 不包含任何方法，因此所有型態都實現了空介面。

### 空介面的用途
- 儲存任意型態的值
- 需要配合型態斷言使用
- 類似於其他語言的 `Object` 或 `Any`

### 避免濫用
- 失去型態安全性
- 需要執行時檢查
- 程式碼可讀性下降

### 正確使用場景
- JSON 解析的未知結構
- 通用容器或函式庫
- 與外部系統互動

In [ ]:
package main

import "fmt"

func main() {
    fmt.Println("=== 空介面基本使用 ===")
    
    var empty interface{}
    
    // 可以存儲任何型態
    empty = 42
    fmt.Printf("empty = %v (type: %T)\n", empty, empty)  // 輸出: empty = 42 (type: int)
    
    empty = "hello"
    fmt.Printf("empty = %v (type: %T)\n", empty, empty)  // 輸出: empty = hello (type: string)
    
    empty = []int{1, 2, 3}
    fmt.Printf("empty = %v (type: %T)\n", empty, empty)  // 輸出: empty = [1 2 3] (type: []int)
    
    fmt.Println("\n=== 空介面切片 ===")
    
    // 存儲不同型態的值
    values := []interface{}{
        42,
        "hello",
        3.14,
        true,
        []int{1, 2, 3},
    }
    
    for i, v := range values {
        fmt.Printf("values[%d] = %v (type: %T)\n", i, v, v)
    }
    
    fmt.Println("\n=== 避免濫用空介面 ===")
    fmt.Println("空介面應該謹慎使用，優先考慮具體型態或合適的介面")
}

## 型態斷言與 type switch

型態斷言讓我們可以從介面中提取具體型態：

### 型態斷言語法
```go
value, ok := interfaceVar.(ConcreteType)
if ok {
    // 型態斷言成功
}
```

### Type Switch 語法
```go
switch v := interfaceVar.(type) {
case string:
    // v 是 string 型態
case int:
    // v 是 int 型態
default:
    // 其他型態
}
```

### 使用場景
- 處理不同型態的值
- 實現多型行為
- 與空介面配合使用

In [ ]:
package main

import "fmt"

func processValue(value interface{}) {
    // 型態斷言
    if str, ok := value.(string); ok {
        fmt.Printf("String: %s (length: %d)\n", str, len(str))
        return
    }
    
    if num, ok := value.(int); ok {
        fmt.Printf("Integer: %d (doubled: %d)\n", num, num*2)
        return
    }
    
    // Type Switch
    switch v := value.(type) {
    case string:
        fmt.Printf("String: %s\n", v)
    case int:
        fmt.Printf("Integer: %d\n", v)
    case float64:
        fmt.Printf("Float: %.2f\n", v)
    case bool:
        fmt.Printf("Boolean: %t\n", v)
    default:
        fmt.Printf("Unknown type: %T, value: %v\n", v, v)
    }
}

func main() {
    fmt.Println("=== 型態斷言與 Type Switch 示例 ===")
    
    values := []interface{}{
        "Hello",
        42,
        3.14159,
        true,
        []int{1, 2, 3},
    }
    
    for _, v := range values {
        processValue(v)
    }
}

## 介面的實行 type switch

使用 type switch 可以根據介面的具體型態執行不同的邏輯，這在處理多種型態時特別有用。

### 實用模式
- 根據型態選擇處理策略
- 型態特定的最佳化
- 漸進式型態檢查

### 最佳實踐
- 優先檢查最常見的型態
- 提供 default 分支
- 避免過深的 type switch 嵌套
- 考慮使用介面方法而非型態檢查

In [ ]:
package main

import "fmt"

type Shape interface {
    Area() float64
}

type Circle struct {
    Radius float64
}

func (c Circle) Area() float64 {
    return 3.14159 * c.Radius * c.Radius
}

type Rectangle struct {
    Width, Height float64
}

func (r Rectangle) Area() float64 {
    return r.Width * r.Height
}

type Triangle struct {
    Base, Height float64
}

func (t Triangle) Area() float64 {
    return 0.5 * t.Base * t.Height
}

// 使用 type switch 進行型態特定的處理
func describeShape(s Shape) {
    fmt.Printf("Area: %.2f, ", s.Area())
    
    // Type switch 根據具體型態執行不同邏輯
    switch shape := s.(type) {
    case Circle:
        fmt.Printf("Type: Circle, Radius: %.2f\n", shape.Radius)
    case Rectangle:
        fmt.Printf("Type: Rectangle, %g x %g\n", shape.Width, shape.Height)
    case Triangle:
        fmt.Printf("Type: Triangle, Base: %g, Height: %g\n", shape.Base, shape.Height)
    default:
        fmt.Printf("Type: Unknown shape (%T)\n", shape)
    }
}

func main() {
    fmt.Println("=== 介面 Type Switch 實戰示例 ===")
    
    shapes := []Shape{
        Circle{Radius: 5},
        Rectangle{Width: 4, Height: 6},
        Triangle{Base: 3, Height: 4},
    }
    
    for _, shape := range shapes {
        describeShape(shape)
    }
}